## Read file PDF For Common name

In [17]:
import re
import tabula.io
import pandas as pd

# ดึงข้อมูลทั้งหมดจาก PDF
pdf_path = "รายชื่อพันธุ์ไม้สามัญและวิทยาศาสตร์2022.pdf"

In [19]:
# pip install tabula-py pdfplumber pandas
import re
import tabula
import pdfplumber
import pandas as pd
from pathlib import Path

PDF_PATH   = "รายชื่อพันธุ์ไม้สามัญและวิทยาศาสตร์2022.pdf"
EFLORA_CSV = "bkf_eflora_species_vol2_16.csv"
OUT_CSV    = "bkf_eflora_with_common_name.csv"
UNMATCHED  = "unmatched_species_from_pdf_join.csv"

KEYWORDS = ["ลำดับ", "ชื่อสามัญ", "วิทยาศาสตร์"]  # ใช้ช่วยหาแถวหัว

def clean_text(s):
    if pd.isna(s): return None
    s = re.sub(r"\s+", " ", str(s)).strip()
    return s if s and s.lower() != "nan" else None

def to_binomial_key(s):
    # ดึง 2 คำแรก: Genus + species (กันเคสมี author/วงเล็บ/ย่อย)
    if not s: return None
    m = re.search(r"([A-Z][a-zA-Z-]+)\s+([a-z\-]+)", s)
    return f"{m.group(1)} {m.group(2)}" if m else s

def collapse_duplicate_columns(df):
    # รวมคอลัมน์ที่ชื่อซ้ำให้เหลือคอลัมน์เดียว (เลือกค่าที่ไม่ว่างทางซ้าย)
    out = pd.DataFrame(index=df.index)
    # รักษาลำดับคอลัมน์เดิม
    for name in dict.fromkeys(df.columns):
        block = df.loc[:, df.columns == name]
        if block.shape[1] == 1:
            out[name] = block.iloc[:, 0]
        else:
            out[name] = block.bfill(axis=1).iloc[:, 0]
    return out

def guess_header_index(df):
    # หาแถวหัวตารางด้วยกฎ: (1) มีคีย์เวิร์ดครบมากสุด (2) non-null เยอะสุด
    best_i, best_score = None, -1
    for i, row in df.iterrows():
        cells = [str(x) for x in row.values]
        text_line = " ".join(cells)
        score = sum(k in text_line for k in KEYWORDS) + (len([x for x in row.values if pd.notna(x)]) * 0.2)
        if score > best_score:
            best_i, best_score = i, score
    return best_i

def extract_from_tabula(pdf_path):
    dfs = tabula.io.read_pdf(
        pdf_path,
        pages="all",
        multiple_tables=True,
        lattice=True,
        pandas_options={'dtype': str}
    )
    if not dfs:
        return pd.DataFrame(columns=["species_scientific_name", "common_name"])
    tables = []
    for t in dfs:
        t = t.copy()
        # ทำความสะอาดเบื้องต้น
        t.columns = [str(c).strip() for c in t.columns]
        # ถ้าหัวเป็น Unnamed เกือบหมด ให้ย้ายไปใช้แถวหัวที่เดา
        hdr_idx = guess_header_index(t)
        if hdr_idx is not None:
            header_row = t.loc[hdr_idx].fillna("").astype(str).str.strip().tolist()
            # forward-fill ชื่อหัวกรณีมีช่องว่าง
            filled = []
            last = ""
            for v in header_row:
                if v:
                    last = v
                    filled.append(v)
                else:
                    filled.append(last)
            # ตั้งหัวใหม่ + ตัดส่วนหัวออก
            t2 = t.loc[hdr_idx+1:].reset_index(drop=True)
            t2.columns = [str(x).strip() for x in filled]
        else:
            t2 = t.copy()

        # รวมคอลัมน์ชื่อซ้ำ
        t2 = collapse_duplicate_columns(t2)
        # normalize ชื่อคอลัมน์
        t2.columns = [str(c).strip() for c in t2.columns]
        # หา column ชื่อวิทยาศาสตร์ (ไม่ใช่ "แบบเต็ม") และชื่อสามัญ
        # เลือกคอลัมน์แรกที่ match เงื่อนไข
        sci_candidates = [c for c in t2.columns if "วิทยาศาสตร์" in c and "แบบเต็ม" not in c]
        com_candidates = [c for c in t2.columns if "ชื่อสามัญ" in c]
        if not sci_candidates or not com_candidates:
            continue
        sci_col = sci_candidates[0]
        com_col = com_candidates[0]
        sub = t2[[sci_col, com_col]].rename(columns={sci_col: "species_scientific_name", com_col: "common_name"})
        sub["species_scientific_name"] = sub["species_scientific_name"].apply(clean_text)
        sub["common_name"] = sub["common_name"].apply(clean_text)
        sub = sub.dropna(subset=["species_scientific_name"])
        tables.append(sub)

    if not tables:
        return pd.DataFrame(columns=["species_scientific_name", "common_name"])
    df = pd.concat(tables, ignore_index=True).drop_duplicates()
    return df

def fallback_pdfplumber(pdf_path):
    # สำรอง: ดึงแบบข้อความดิบ (ไม่ใช้ Java)
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            for line in text.split("\n"):
                # รูปแบบคร่าว ๆ: <ลำดับ> <ชื่อสามัญ> ..... <ชื่อวิทยาศาสตร์>
                # ต้องปรับ regex ตามไฟล์จริงถ้ายังไม่ match
                m = re.match(r"^\s*\d+\s+([ก-๙A-Za-z\-\s\(\)\/\.\,]+?)\s{2,}([A-Z][A-Za-z\-]+\s+[a-z\-]+.*)$", line)
                if m:
                    rows.append({
                        "common_name": clean_text(m.group(1)),
                        "species_scientific_name": clean_text(m.group(2))
                    })
    return pd.DataFrame(rows).dropna().drop_duplicates()

# --------- RUN EXTRACT ---------
df_common = extract_from_tabula(PDF_PATH)
if df_common.empty:
    print("⚠️ Tabula จับตารางไม่ได้ ลองใช้ pdfplumber สำรอง…")
    df_common = fallback_pdfplumber(PDF_PATH)

# ทำ key ไบนอเมียล
df_common["species_key"] = df_common["species_scientific_name"].apply(to_binomial_key)

# โหลด e-Flora แล้ว join
df_e = pd.read_csv(EFLORA_CSV)
df_e["species_key"] = df_e["species_scientific_name"].apply(to_binomial_key)

df_merge = df_e.merge(
    df_common[["species_key", "common_name"]].drop_duplicates(),
    on="species_key",
    how="left"
).drop(columns=["species_key"])

# เซฟผลลัพธ์ + รายการที่ยังไม่มีชื่อสามัญ
df_merge.to_csv(OUT_CSV, index=False)
df_merge[df_merge["common_name"].isna()][["species_scientific_name","family_name","genus_name","species_url"]].to_csv(UNMATCHED, index=False)

print(f"✅ รวมข้อมูลเสร็จสิ้น → {OUT_CSV} (ทั้งหมด {len(df_merge)} แถว)")
print(f"📄 รายการที่ยังไม่มีชื่อสามัญ → {UNMATCHED} (จำนวน {df_merge['common_name'].isna().sum()} แถว)")

/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_5047/2978806502.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[name] = block.bfill(axis=1).iloc[:, 0]
/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_5047/2978806502.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[name] = block.bfill(axis=1).iloc[:, 0]
/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_5047/2978806502.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result

⚠️ Tabula จับตารางไม่ได้ ลองใช้ pdfplumber สำรอง…


KeyError: 'species_scientific_name'

In [16]:
# ดึงทุกตารางในทุกหน้า (ใช้ lattice เพื่อเก็บโครงตารางแน่นอน)
dfs = tabula.io.read_pdf(pdf_path, pages="all", multiple_tables=True, lattice=True)

# รวมทุกตารางเข้าด้วยกัน
df_pdf = pd.concat(dfs, ignore_index=True)
df_pdf

,Unnamed: 0,รายชื่อพนัธุ์ไม้สามัญและวทิยาศาสตร์,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,ลาํดบ,NaN,ชื่อสามญ,NaN,ชื่อพืน้เมือง,NaN,ชื่อวทิยาศาสตร์แบบเตม,NaN,ชื่อวทิยาศาสตร์,NaN,ชื่อคนค้นพบ,NaN,วงศ์,NaN
1,NaN,1,NaN,กก,NaN,กก,NaN,Cyperus imbricatus Retz.,NaN,Cyperus imbricatus,NaN,Retz.,NaN,CYPERACEAE,NaN
2,NaN,2,NaN,กก,NaN,กก,NaN,Cyperus exaltatus Retz.,NaN,Cyperus exaltatus,NaN,Retz.,NaN,CYPERACEAE,NaN
3,NaN,3,NaN,กกกร่อย,NaN,กกกร่อย,NaN,Schoenoplectus littoralis (Schrad.) Palla\rsub...,NaN,Schoenoplectus littoralis subsp. thermalis,NaN,(Trab.) S. S. Hooper,NaN,CYPERACEAE,NaN
4,NaN,4,NaN,กกกระจาย,NaN,กกกระจาย,NaN,Cyperus elatus L.,NaN,Cyperus elatus,NaN,L.,NaN,CYPERACEAE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13264,NaN,12192,NaN,Ziziphus sp.,NaN,NaN,NaN,NaN,NaN,Ziziphus sp.,NaN,NaN,NaN,RHAMNACEAE,NaN
13265,NaN,12193,NaN,Zollingeria sp.,NaN,NaN,NaN,NaN,NaN,Zollingeria sp.,NaN,NaN,NaN,SAPINDACEAE,NaN
13266,NaN,12194,NaN,Zornia sp.,NaN,NaN,NaN,NaN,NaN,Zornia sp.,NaN,NaN,NaN,FABACEAE,NaN
13267,NaN,12195,NaN,Zoysia sp.,NaN,NaN,NaN,NaN,NaN,Zoysia sp.,NaN,NaN,NaN,POACEAE,NaN


In [14]:
# ดึงทุกตารางในทุกหน้า (ใช้ lattice เพื่อเก็บโครงตารางแน่นอน)
dfs = tabula.io.read_pdf(pdf_path, pages="all", multiple_tables=True, lattice=True)

# รวมทุกตารางเข้าด้วยกัน
df_pdf = pd.concat(dfs, ignore_index=True)

# ลบช่องว่างในชื่อคอลัมน์
df_pdf.columns = [c.strip() for c in df_pdf.columns]

# เลือกเฉพาะคอลัมน์ที่ต้องใช้ และ rename ให้ตรงกับ e-Flora
df_common = df_pdf[["ชื่อวิทยาศาสตร์", "ชื่อสามัญ"]].rename(
    columns={"ชื่อวิทยาศาสตร์": "species_scientific_name", "ชื่อสามัญ": "common_name"}
)

KeyError: "None of [Index(['ชื่อวิทยาศาสตร์', 'ชื่อสามัญ'], dtype='object')] are in the [columns]"

In [15]:
print(df_pdf.columns.tolist())

['Unnamed: 0', 'รายชื่อพนัธุ์ไม้สามัญและวทิยาศาสตร์', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']


In [2]:
# show sample
print(df_common.head())

NameError: name 'df_common' is not defined

# Appendix

In [4]:
import pandas as pd
import requests, re, time, random
from bs4 import BeautifulSoup

IN_CSV  = "bkf_eflora_species_new.csv"
OUT_CSV = "bkf_eflora_species_test_wfo.csv"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,th;q=0.8",
    "Referer": "https://botany.dnp.go.th/",
}

WFO_RE = re.compile(r"wfo-\d{7,}")  # กันเหนียว 7 หลักขึ้นไป

def get_html(url, tries=3):
    for _ in range(tries):
        try:
            r = requests.get(url, headers=HEADERS, timeout=20)
            if r.status_code == 200 and r.text:
                return r.text
        except requests.RequestException:
            pass
        time.sleep(1.0 + random.random())
    return None

from typing import Union

def extract_wfo_id(html: str) -> Union[str, None]:
    soup = BeautifulSoup(html, "html.parser")

    # 1) ลองหา <a> ที่มีสีเขียว (อย่าเท่ากับเป๊ะ ใช้ *contains*)
    for a in soup.select("a[style*='color:#279275']"):
        # จาก text
        m = WFO_RE.search(a.get_text(" ", strip=True))
        if m:
            return m.group(0)
        # จาก href
        m = WFO_RE.search(a.get("href", ""))
        if m:
            return m.group(0)

    # 2) เผื่อ style ไม่ตรง ให้หา href โดนๆ ก่อน
    for a in soup.select("a[href*='worldfloraonline.org']"):
        m = WFO_RE.search((a.get_text(" ", strip=True)) or "")
        if m:
            return m.group(0)
        m = WFO_RE.search(a.get("href", ""))
        if m:
            return m.group(0)

    # 3) สุดท้าย สแกนทั้งหน้า
    m = WFO_RE.search(soup.get_text(" ", strip=True))
    if m:
        return m.group(0)

    return None

# --- ทดสอบ 10 แถวแรก ---
df = pd.read_csv(IN_CSV)
df_test = df.head(10).copy()
df_test["id_wfo"] = pd.NA

for i, row in df_test.iterrows():
    url = str(row.get("species_url", "")).strip()
    if not url:
        continue

    html = get_html(url)
    if not html:
        print(f"[{i+1}/10] fetch fail: {row.get('species_scientific_name')}")
        continue

    wfo_id = extract_wfo_id(html)
    df_test.at[i, "id_wfo"] = wfo_id
    print(f"[{i+1}/10] {row.get('species_scientific_name')}: {wfo_id}")

    time.sleep(random.uniform(0.9, 1.6))

df_test
#df_test.to_csv(OUT_CSV, index=False)
#print(f"✅ Saved preview → {OUT_CSV}")

[1/10] Myriophyllum siamense (Craib) Tardieu: None
[2/10] Myriophyllum tetrandrum Roxb.: None
[3/10] Myriophyllum siamense (Craib) Tardieu: None
[4/10] Myriophyllum tetrandrum Roxb.: None
[5/10] Haloragis micrantha (Thunb.) R.Br. ex Sieb. & Zucc.: None
[6/10] Haloragis micrantha (Thunb.) R.Br. ex Sieb. & Zucc.: None
[7/10] Rhizophora apiculata Blume: None
[8/10] Rhizophora mucronata Poir.: None
[9/10] Rhizophora apiculata Blume: None
[10/10] Rhizophora mucronata Poir.: None


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,id_wfo
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,None
1,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:36.633398+00:00,Myriophyllum tetrandrum Roxb.,None
2,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-19T18:17:39.543048+00:00,Myriophyllum siamense (Craib) Tardieu,None
3,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-19T18:17:42.567178+00:00,Myriophyllum tetrandrum Roxb.,None
4,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:50.049475+00:00,Gonocarpus micranthus,None
5,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-19T18:17:53.030223+00:00,Gonocarpus micranthus,None
6,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora apiculata Blume,This is currently accepted.,CENTRAL: Chon Buri (Si Racha); SOUTH-EASTERN: ...,In tropical SE Asia throughout Malesia (type) ...,Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:15.874773+00:00,Rhizophora apiculata Blume,None
7,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora mucronata Poir.,This is currently accepted.,SOUTH-EASTERN: Chanthaburi; SOUTH-WESTERN: Pra...,"In the Old World tropics, occurring from the c...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:19.615290+00:00,Rhizophora mucronata Poir.,None
8,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora apicu